In [1]:
from model.model import IndexModel, SyntacticIndex, SemanticIndex

import json
from utils.utils import loadCorpus, saveCorpus, load_dictprop, getTmpDictProp, save_dataset, load_dataset, prepare_corpus, vectorize_data, showEval, evaluate_models
import pandas as pd


from collections import Counter, defaultdict

from numpy import mean


from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

%load_ext autoreload
%autoreload 2
n_core = 6

In [2]:
project = f'../data/Q5'

with open(f"{project}/entity_types.json", 'r', encoding='utf-8') as f:
    entitytype = json.load(f)


list_entitytype = list(entitytype.keys())

print(project)

dict_prop = load_dictprop(project)
tmp_dict = getTmpDictProp(dict_prop=dict_prop)

dict_prop, tmp_dict

../data/Q5


([{'type': 'Q5',
   'name': 'person',
   'props': {'P19': 'placeOfBirth',
    'P569': 'dateOfBirth',
    'P570': 'dateOfDeath',
    'P26': 'spouse',
    'P106': 'occupation',
    'P69': 'educatedAt',
    'Other': 'Other'}}],
 {'P19': 'placeOfBirth',
  'P569': 'dateOfBirth',
  'P570': 'dateOfDeath',
  'P26': 'spouse',
  'P106': 'occupation',
  'P69': 'educatedAt',
  'Other': 'Other'})

## Prepare dataset

In [3]:
clean = True
corpus = loadCorpus(project, clean=clean)
len(corpus), corpus[0]

(1066,
 {'id': 'Q10007',
  'content': [{'sent': 'En 1519, il mène une autre expédition, dans le Jiangxi, pour mater un prince rebelle, mais découvre en arrivant dans cette province que la révolte a déjà été vaincue.',
    'sent_i': 0,
    'props': [{'prop': 'Other',
      'sent': 'En 1519, il mène une autre expédition, dans le Jiangxi, pour mater un prince rebelle, mais découvre en arrivant dans cette province que la révolte a déjà été vaincue.',
      'source': 'NO-SOURCE',
      'target': 'NO-TARGET',
      'sdpgraphs': [{'sourceNode': None,
        'targetNode': None,
        'sdpgraph': <networkx.classes.digraph.DiGraph at 0x7febecd318b0>}]}],
    'head': 4,
    'graph': <networkx.classes.digraph.DiGraph at 0x7febecd315e0>}]})

In [4]:
params = {
    "corpus": corpus,
    "train_size": .7,
    "dev_size": .5,
    "clean": True
    # "maxsize": 200000
}
data = prepare_corpus(**params)
# print(len(list_graphs))
# list_graphs[0]
try:
    print(len(data['X_train']), len(data['X_dev']), len(data['X_test']))
except:
    print(len(data['X_train']), len(data['y_train']))


2182 544 545


In [7]:
cc_dict = defaultdict(int)
limit = 303
filtered_data = {
    "X_train": [],
    "y_train": []
}
for i, y in enumerate(data['y_train']):
    x = data['X_train'][i]
    # y = tmp_dict[y]
    if cc_dict[y] < limit:
        cc_dict[y] += 1
        filtered_data['X_train'].append(x)
        filtered_data['y_train'].append(y)

data = filtered_data


In [5]:
c = Counter([tmp_dict[y] for y in data['y_train']])
print(pd.Series(dict(c.most_common())).to_frame().to_latex())

\begin{tabular}{lr}
\toprule
{} &    0 \\
\midrule
spouse       &  386 \\
dateOfBirth  &  373 \\
educatedAt   &  367 \\
dateOfDeath  &  355 \\
placeOfBirth &  353 \\
occupation   &  348 \\
\bottomrule
\end{tabular}



In [6]:
c = Counter([tmp_dict[y] for y in data['y_dev']])
print(pd.Series(dict(c.most_common())).to_frame().to_latex())

\begin{tabular}{lr}
\toprule
{} &   0 \\
\midrule
spouse       &  92 \\
Other        &  83 \\
occupation   &  78 \\
dateOfBirth  &  77 \\
educatedAt   &  76 \\
placeOfBirth &  70 \\
dateOfDeath  &  68 \\
\bottomrule
\end{tabular}



In [7]:
c = Counter([tmp_dict[y] for y in data['y_test']])
print(pd.Series(dict(c.most_common())).to_frame().to_latex())

\begin{tabular}{lr}
\toprule
{} &   0 \\
\midrule
Other        &  95 \\
educatedAt   &  81 \\
occupation   &  80 \\
placeOfBirth &  77 \\
dateOfBirth  &  73 \\
dateOfDeath  &  72 \\
spouse       &  67 \\
\bottomrule
\end{tabular}



In [8]:
evalpath = f'../data/Q5-clean'
clean = True
evalcorpus = loadCorpus(evalpath, clean=clean)

params = {
    "corpus": evalcorpus,
    "train_size": .7,
    "dev_size": .5,
    "clean": True
    # "maxsize": 200000
}
evaldata = prepare_corpus(**params)
try:
    print(len(evaldata['X_train']), len(evaldata['X_dev']), len(evaldata['X_test']))
except:
    print(len(evaldata['X_train']), len(evaldata['y_train']))

579 186 186


In [9]:
c = Counter([tmp_dict[y] for y in evaldata['y_train']])
print(pd.Series(dict(c.most_common())).to_frame().to_latex())

\begin{tabular}{lr}
\toprule
{} &    0 \\
\midrule
occupation   &  266 \\
placeOfBirth &  117 \\
dateOfBirth  &   88 \\
dateOfDeath  &   60 \\
spouse       &   28 \\
educatedAt   &   20 \\
\bottomrule
\end{tabular}



In [10]:
c = Counter([tmp_dict[y] for y in evaldata['y_dev']])
print(pd.Series(dict(c.most_common())).to_frame().to_latex())

\begin{tabular}{lr}
\toprule
{} &   0 \\
\midrule
Other        &  62 \\
occupation   &  57 \\
placeOfBirth &  21 \\
dateOfBirth  &  21 \\
dateOfDeath  &  17 \\
spouse       &   6 \\
educatedAt   &   2 \\
\bottomrule
\end{tabular}



In [11]:
c = Counter([tmp_dict[y] for y in evaldata['y_test']])
print(pd.Series(dict(c.most_common())).to_frame().to_latex())

\begin{tabular}{lr}
\toprule
{} &   0 \\
\midrule
Other        &  64 \\
occupation   &  54 \\
dateOfBirth  &  27 \\
placeOfBirth &  16 \\
spouse       &  14 \\
dateOfDeath  &   8 \\
educatedAt   &   3 \\
\bottomrule
\end{tabular}



## Train index

In [12]:
syntactic_index_params = {
    "list_graphs": data['X_train'],
    "anchor_textvalue": ['text', 'pos'],
    "graphkey": 'sdpgraph',
    "propkey": 'prop',
    # "support": 2,
    "dict_prop": dict_prop,
    "savepath": project
}
syntacticIndex = SyntacticIndex()
syntacticIndex.trainSyntacticIndex(**syntactic_index_params)
syntacticIndex.syntacticIndex

{' _SPACE': [{'graph': <networkx.classes.digraph.DiGraph at 0x7febcf7593a0>,
   'size': 4,
   'props': [{'name': 'dateOfBirth', 'support': 1}],
   'ambiguous': 0,
   'source_types': ['Q5'],
   'source_nodes': [0],
   'target_types': ['time'],
   'target_nodes': [14],
   'ner_rules': {'dateOfBirth': {'source_type': 'Q5', 'target_type': 'time'}},
   'i': 1},
  {'graph': <networkx.classes.digraph.DiGraph at 0x7febcf7593d0>,
   'size': 5,
   'props': [{'name': 'dateOfDeath', 'support': 1}],
   'ambiguous': 0,
   'source_types': ['Q5'],
   'source_nodes': [0],
   'target_types': ['time'],
   'target_nodes': [26],
   'ner_rules': {'dateOfDeath': {'source_type': 'Q5', 'target_type': 'time'}},
   'i': 2},
  {'graph': <networkx.classes.digraph.DiGraph at 0x7feba835c2e0>,
   'size': 5,
   'props': [{'name': 'dateOfDeath', 'support': 1}],
   'ambiguous': 0,
   'source_types': ['Q5'],
   'source_nodes': [0],
   'target_types': ['time'],
   'target_nodes': [45],
   'ner_rules': {'dateOfDeath': {'so

In [26]:

print('Unique anchors', len(syntacticIndex.syntacticIndex.keys()))
print('Unique patterns', len([y['graph'] for x in syntacticIndex.syntacticIndex.values() for y in x]))
print('Ambiguous', sum([y['ambiguous'] for x in syntacticIndex.syntacticIndex.values() for y in x ]))
print('Mean graph', mean([len(x) for x in syntacticIndex.syntacticIndex.values()]))

Unique anchors 497
Unique patterns 1024
Ambiguous 3
Mean graph 2.0603621730382295


In [25]:
for x in syntacticIndex.syntacticIndex.values():
    print(x)


[{'graph': <networkx.classes.digraph.DiGraph object at 0x7febcf7593a0>, 'size': 4, 'props': [{'name': 'dateOfBirth', 'support': 1}], 'ambiguous': 0, 'source_types': ['Q5'], 'source_nodes': [0], 'target_types': ['time'], 'target_nodes': [14], 'ner_rules': {'dateOfBirth': {'source_type': 'Q5', 'target_type': 'time'}}, 'i': 1}, {'graph': <networkx.classes.digraph.DiGraph object at 0x7febcf7593d0>, 'size': 5, 'props': [{'name': 'dateOfDeath', 'support': 1}], 'ambiguous': 0, 'source_types': ['Q5'], 'source_nodes': [0], 'target_types': ['time'], 'target_nodes': [26], 'ner_rules': {'dateOfDeath': {'source_type': 'Q5', 'target_type': 'time'}}, 'i': 2}, {'graph': <networkx.classes.digraph.DiGraph object at 0x7feba835c2e0>, 'size': 5, 'props': [{'name': 'dateOfDeath', 'support': 1}], 'ambiguous': 0, 'source_types': ['Q5'], 'source_nodes': [0], 'target_types': ['time'], 'target_nodes': [45], 'ner_rules': {'dateOfDeath': {'source_type': 'Q5', 'target_type': 'time'}}, 'i': 3}, {'graph': <networkx.c

In [15]:
syntactic_index_params = {
    "list_graphs": evaldata['X_train'],
    "anchor_textvalue": ['text', 'pos'],
    "graphkey": 'sdpgraph',
    "propkey": 'prop',
    # "support": 2,
    "dict_prop": dict_prop,
    "savepath": evalpath
}
clean_syntacticIndex = SyntacticIndex()
clean_syntacticIndex.trainSyntacticIndex(**syntactic_index_params)
clean_syntacticIndex.syntacticIndex

{' _SPACE': [{'graph': <networkx.classes.digraph.DiGraph at 0x7febac7ce3a0>,
   'size': 7,
   'props': [{'name': 'placeOfBirth', 'support': 1}],
   'ambiguous': 0,
   'source_types': ['Q5'],
   'source_nodes': [0],
   'target_types': ['Q30974'],
   'target_nodes': [22],
   'ner_rules': {'placeOfBirth': {'source_type': 'Q5',
     'target_type': 'Q30974'}},
   'i': 1}],
 '10_NUM': [{'graph': <networkx.classes.digraph.DiGraph at 0x7febabcd8e50>,
   'size': 3,
   'props': [{'name': 'dateOfBirth', 'support': 1}],
   'ambiguous': 0,
   'source_types': ['Q5'],
   'source_nodes': [20],
   'target_types': ['time'],
   'target_nodes': [14],
   'ner_rules': {'dateOfBirth': {'source_type': 'Q5', 'target_type': 'time'}},
   'i': 1},
  {'graph': <networkx.classes.digraph.DiGraph at 0x7febabcd8df0>,
   'size': 3,
   'props': [{'name': 'placeOfBirth', 'support': 1}],
   'ambiguous': 0,
   'source_types': ['Q5'],
   'source_nodes': [20],
   'target_types': ['Q3449906'],
   'target_nodes': [8],
   'ner_

In [27]:

print('Unique anchors', len(clean_syntacticIndex.syntacticIndex.keys()))
print('Unique patterns', len([y['graph'] for x in clean_syntacticIndex.syntacticIndex.values() for y in x]))
print('Ambiguous', sum([y['ambiguous'] for x in clean_syntacticIndex.syntacticIndex.values() for y in x ]))
print('Mean graph', mean([len(x) for x in clean_syntacticIndex.syntacticIndex.values()]))

Unique anchors 173
Unique patterns 335
Ambiguous 0
Mean graph 1.9364161849710984


# Classification
## Index classifier

In [17]:
semantic_index_params = {
    "list_graphs": data['X_train'],
    "textvalue":  syntacticIndex.syntacticIndexParams['anchor_textvalue'],
    "dict_prop": dict_prop,
    "savepath": project

}
base_semanticIndex = SemanticIndex()
base_semanticIndex.trainSemanticIndex(**semantic_index_params)
base_semanticIndex.semanticIndex

CONCEPT-INDEX,occupation,placeOfBirth,spouse,dateOfBirth,dateOfDeath,educatedAt
né_VERB,0.002464,0.327256,0.006602,0.731058,0.598672,0.000000
27_NUM,0.000000,0.000000,0.000000,0.575312,0.817934,0.000000
mort_VERB,0.000000,0.003850,0.000000,0.000000,0.999993,0.000000
succès_NOUN,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
peut_VERB,0.307395,0.862640,0.274567,0.000000,0.000000,0.293214
...,...,...,...,...,...,...
auditions_NOUN,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
attribue_VERB,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Conservatoire_NOUN,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
passé_VERB,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [18]:
print(base_semanticIndex.semanticIndex.round(decimals=3).head(n=10).to_latex())

\begin{tabular}{lrrrrrr}
\toprule
CONCEPT-INDEX &  occupation &  placeOfBirth &  spouse &  dateOfBirth &  dateOfDeath &  educatedAt \\
\midrule
né\_VERB        &       0.002 &         0.327 &   0.007 &        0.731 &        0.599 &       0.000 \\
27\_NUM         &       0.000 &         0.000 &   0.000 &        0.575 &        0.818 &       0.000 \\
mort\_VERB      &       0.000 &         0.004 &   0.000 &        0.000 &        1.000 &       0.000 \\
succès\_NOUN    &       0.000 &         1.000 &   0.000 &        0.000 &        0.000 &       0.000 \\
peut\_VERB      &       0.307 &         0.863 &   0.275 &        0.000 &        0.000 &       0.293 \\
saper\_VERB     &       0.000 &         1.000 &   0.000 &        0.000 &        0.000 &       0.000 \\
autorité\_NOUN  &       0.000 &         1.000 &   0.000 &        0.000 &        0.000 &       0.000 \\
église\_NOUN    &       0.000 &         1.000 &   0.000 &        0.000 &        0.000 &       0.000 \\
entamé\_VERB    &       0.000 & 

In [19]:
semantic_index_params = {
    "list_graphs": evaldata['X_train'],
    "textvalue":  syntacticIndex.syntacticIndexParams['anchor_textvalue'],
    "dict_prop": dict_prop,
    "savepath": evalpath

}
clean_semanticIndex = SemanticIndex()
clean_semanticIndex.trainSemanticIndex(**semantic_index_params)
clean_semanticIndex.semanticIndex

CONCEPT-INDEX,occupation,placeOfBirth,spouse,dateOfBirth,dateOfDeath,educatedAt
épouse_VERB,0.0,0.000000,1.0,0.000000,0.000000,0.0
maison_NOUN,0.0,1.000000,0.0,0.000000,0.000000,0.0
naît_VERB,0.0,0.697524,0.0,0.716561,0.000000,0.0
23_NUM,0.0,0.000000,0.0,0.968241,0.250018,0.0
missionnaire_ADJ,1.0,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...
prolongent_VERB,1.0,0.000000,0.0,0.000000,0.000000,0.0
travail_NOUN,1.0,0.000000,0.0,0.000000,0.000000,0.0
dévoile_VERB,1.0,0.000000,0.0,0.000000,0.000000,0.0
forme_NOUN,1.0,0.000000,0.0,0.000000,0.000000,0.0


In [20]:
print(clean_semanticIndex.semanticIndex.round(decimals=3).head(n=10).to_latex())

\begin{tabular}{lrrrrrr}
\toprule
CONCEPT-INDEX &  occupation &  placeOfBirth &  spouse &  dateOfBirth &  dateOfDeath &  educatedAt \\
\midrule
épouse\_VERB       &       0.000 &         0.000 &   1.000 &        0.000 &        0.000 &         0.0 \\
maison\_NOUN       &       0.000 &         1.000 &   0.000 &        0.000 &        0.000 &         0.0 \\
naît\_VERB         &       0.000 &         0.698 &   0.000 &        0.717 &        0.000 &         0.0 \\
23\_NUM            &       0.000 &         0.000 &   0.000 &        0.968 &        0.250 &         0.0 \\
missionnaire\_ADJ  &       1.000 &         0.000 &   0.000 &        0.000 &        0.000 &         0.0 \\
horticulteur\_NOUN &       1.000 &         0.000 &   0.000 &        0.000 &        0.000 &         0.0 \\
né\_VERB           &       0.014 &         0.628 &   0.015 &        0.624 &        0.464 &         0.0 \\
15\_NUM            &       0.000 &         0.000 &   0.000 &        0.979 &        0.202 &         0.0 \\
conféren

## Evaluation

In [28]:
models = {
    "base_semanticIndex": IndexModel(extractor=syntacticIndex, classifier=base_semanticIndex),
    "clean_semanticIndex": IndexModel(extractor=clean_syntacticIndex, classifier=clean_semanticIndex),

}
list_thresh = [0, .1, .2, .3, .4, .5, .6, .7, .8, .9]
dev_true_y = [tmp_dict[x] for x in data['y_dev']]
test_true_y = [tmp_dict[x] for x in data['y_test']]
# upper_bound = 1

eval_dict = evaluate_models(models=models, evaldata=data, dev_true_y=dev_true_y, test_true_y=test_true_y, list_thresh=list_thresh)

/Users/nicolasgutehrle/opt/anaconda3/envs/corpus/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nicolasgutehrle/opt/anaconda3/envs/corpus/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nicolasgutehrle/opt/anaconda3/envs/corpus/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len

In [29]:
for key in eval_dict.keys():
    print(key)
    eval = showEval(eval_dict, key)
    print(eval.round(decimals=3).to_latex())

base_semanticIndex
\begin{tabular}{lrrrrrr}
\toprule
{} & \multicolumn{3}{l}{Dev} & \multicolumn{3}{l}{Test} \\
{} &      P &      R &     F1 &      P &      R &     F1 \\
\midrule
0.0  &  0.824 &  0.272 &  0.402 &  0.816 &  0.254 &  0.377 \\
0.1  &  0.824 &  0.265 &  0.394 &  0.816 &  0.252 &  0.375 \\
0.2  &  0.824 &  0.259 &  0.387 &  0.816 &  0.252 &  0.375 \\
0.3  &  0.824 &  0.259 &  0.387 &  0.823 &  0.250 &  0.373 \\
0.4  &  0.824 &  0.235 &  0.353 &  0.819 &  0.239 &  0.356 \\
0.5  &  0.828 &  0.222 &  0.336 &  0.818 &  0.235 &  0.351 \\
0.6  &  0.687 &  0.204 &  0.307 &  0.785 &  0.216 &  0.320 \\
0.7  &  0.687 &  0.188 &  0.290 &  0.784 &  0.209 &  0.312 \\
0.8  &  0.687 &  0.177 &  0.274 &  0.831 &  0.197 &  0.297 \\
0.9  &  0.554 &  0.146 &  0.227 &  0.691 &  0.162 &  0.242 \\
10.0 &  0.756 &  0.223 &  0.336 &  0.800 &  0.226 &  0.338 \\
\bottomrule
\end{tabular}

clean_semanticIndex
\begin{tabular}{lrrrrrr}
\toprule
{} & \multicolumn{3}{l}{Dev} & \multicolumn{3}{l}{Test} 

In [30]:
for k in eval_dict.keys():
    print(k)
    print(eval_dict[k][0]['dev']['report'])

base_semanticIndex
              precision    recall  f1-score   support

       Other       0.00      0.00      0.00         0
 dateOfBirth       1.00      0.26      0.41        77
 dateOfDeath       1.00      0.25      0.40        68
  educatedAt       0.79      0.41      0.54        76
  occupation       0.97      0.44      0.60        78
placeOfBirth       1.00      0.30      0.46        70
      spouse       1.00      0.25      0.40        92

    accuracy                           0.32       461
   macro avg       0.82      0.27      0.40       461
weighted avg       0.96      0.32      0.47       461

clean_semanticIndex
              precision    recall  f1-score   support

       Other       0.00      0.00      0.00         0
 dateOfBirth       1.00      0.16      0.27        77
 dateOfDeath       1.00      0.21      0.34        68
  educatedAt       0.50      0.03      0.05        76
  occupation       0.91      0.13      0.22        78
placeOfBirth       1.00      0.24      

In [31]:
for k in eval_dict.keys():
    print(k)
    print(eval_dict[k][0]['test']['report'])

base_semanticIndex
              precision    recall  f1-score   support

       Other       0.00      0.00      0.00         0
 dateOfBirth       0.95      0.25      0.39        73
 dateOfDeath       1.00      0.15      0.27        72
  educatedAt       0.94      0.41      0.57        81
  occupation       0.97      0.47      0.64        80
placeOfBirth       0.94      0.19      0.32        77
      spouse       0.91      0.30      0.45        67

    accuracy                           0.30       450
   macro avg       0.82      0.25      0.38       450
weighted avg       0.95      0.30      0.44       450

clean_semanticIndex
              precision    recall  f1-score   support

       Other       0.00      0.00      0.00         0
 dateOfBirth       1.00      0.18      0.30        73
 dateOfDeath       1.00      0.08      0.15        72
  educatedAt       1.00      0.01      0.02        81
  occupation       0.72      0.23      0.34        80
placeOfBirth       0.92      0.14      

In [32]:

for k in eval_dict.keys():
    print(k)
    devPred = eval_dict[k][0]['devPred']
    devTrue = eval_dict[k][0]['trueDev']

    l_rules = []
    for x, y in zip(devPred, devTrue):
        if x['pred'] != y and y !='Other':
            l_rules.append(x['rule'])
            # print(x['pred'], y,  x['rule'], x['score'])

    c = Counter(l_rules)
    print(c.most_common())


# print()

base_semanticIndex
[('noAnchorMatch', 209), ('noPatternMatch', 97), ('semantic', 9)]
clean_semanticIndex
[('noAnchorMatch', 312), ('noPatternMatch', 75), ('semantic', 3)]


In [33]:
for k in eval_dict.keys():
    print(k)
    testPred = eval_dict[k][0]['testPred']
    trueTest = eval_dict[k][0]['trueTest']

    l_rules = []
    for x, y in zip(testPred, trueTest):
        if x['pred'] != y and y !='Other':
            l_rules.append(x['rule'])
            # print(x['pred'], y,  x['rule'], x['score'])

    c = Counter(l_rules)
    print(c.most_common())


# print()

base_semanticIndex
[('noAnchorMatch', 196), ('noPatternMatch', 112), ('semantic', 7)]
clean_semanticIndex
[('noAnchorMatch', 304), ('noPatternMatch', 80), ('semantic', 9)]


In [34]:
models = {
    "base_semanticIndex": IndexModel(extractor=syntacticIndex, classifier=base_semanticIndex),
    "clean_semanticIndex": IndexModel(extractor=clean_syntacticIndex, classifier=clean_semanticIndex),

}
list_thresh = [0, .1, .2, .3, .4, .5, .6, .7, .8, .9]
dev_true_y = [tmp_dict[x] for x in evaldata['y_dev']]
test_true_y = [tmp_dict[x] for x in evaldata['y_test']]
# upper_bound = 1

clean_eval_dict = evaluate_models(models=models, evaldata=evaldata, dev_true_y=dev_true_y, test_true_y=test_true_y, list_thresh=list_thresh)

/Users/nicolasgutehrle/opt/anaconda3/envs/corpus/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nicolasgutehrle/opt/anaconda3/envs/corpus/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nicolasgutehrle/opt/anaconda3/envs/corpus/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len

In [35]:
for key in clean_eval_dict.keys():
    print(key)
    eval = showEval(clean_eval_dict, key)
    print(eval.round(decimals=3).to_latex())

base_semanticIndex
\begin{tabular}{lrrrrrr}
\toprule
{} & \multicolumn{3}{l}{Dev} & \multicolumn{3}{l}{Test} \\
{} &      P &      R &     F1 &      P &      R &     F1 \\
\midrule
0.0  &  0.771 &  0.393 &  0.443 &  0.750 &  0.311 &  0.378 \\
0.1  &  0.771 &  0.346 &  0.388 &  0.755 &  0.301 &  0.373 \\
0.2  &  0.771 &  0.339 &  0.378 &  0.755 &  0.295 &  0.365 \\
0.3  &  0.771 &  0.339 &  0.378 &  0.755 &  0.295 &  0.365 \\
0.4  &  0.771 &  0.332 &  0.371 &  0.755 &  0.286 &  0.352 \\
0.5  &  0.771 &  0.318 &  0.354 &  0.755 &  0.286 &  0.352 \\
0.6  &  0.771 &  0.290 &  0.316 &  0.612 &  0.248 &  0.296 \\
0.7  &  0.771 &  0.283 &  0.304 &  0.612 &  0.248 &  0.296 \\
0.8  &  0.629 &  0.268 &  0.282 &  0.612 &  0.248 &  0.296 \\
0.9  &  0.486 &  0.232 &  0.224 &  0.476 &  0.201 &  0.232 \\
10.0 &  0.729 &  0.314 &  0.344 &  0.684 &  0.272 &  0.331 \\
\bottomrule
\end{tabular}

clean_semanticIndex
\begin{tabular}{lrrrrrr}
\toprule
{} & \multicolumn{3}{l}{Dev} & \multicolumn{3}{l}{Test} 

In [36]:
for k in clean_eval_dict.keys():
    print(k)
    print(clean_eval_dict[k][0]['dev']['report'])

base_semanticIndex
              precision    recall  f1-score   support

       Other       0.00      0.00      0.00         0
 dateOfBirth       1.00      0.19      0.32        21
 dateOfDeath       1.00      0.47      0.64        17
  educatedAt       0.40      1.00      0.57         2
  occupation       1.00      0.21      0.35        57
placeOfBirth       1.00      0.38      0.55        21
      spouse       1.00      0.50      0.67         6

    accuracy                           0.30       124
   macro avg       0.77      0.39      0.44       124
weighted avg       0.99      0.30      0.44       124

clean_semanticIndex
              precision    recall  f1-score   support

       Other       0.00      0.00      0.00         0
 dateOfBirth       1.00      0.14      0.25        21
 dateOfDeath       0.75      0.35      0.48        17
  educatedAt       0.00      0.00      0.00         2
  occupation       1.00      0.47      0.64        57
placeOfBirth       1.00      0.29      

In [37]:
for k in clean_eval_dict.keys():
    print(k)
    print(clean_eval_dict[k][0]['test']['report'])

base_semanticIndex
              precision    recall  f1-score   support

       Other       0.00      0.00      0.00         0
 dateOfBirth       1.00      0.22      0.36        27
 dateOfDeath       1.00      0.38      0.55         8
  educatedAt       0.25      0.67      0.36         3
  occupation       1.00      0.30      0.46        54
placeOfBirth       1.00      0.19      0.32        16
      spouse       1.00      0.43      0.60        14

    accuracy                           0.30       122
   macro avg       0.75      0.31      0.38       122
weighted avg       0.98      0.30      0.44       122

clean_semanticIndex
              precision    recall  f1-score   support

       Other       0.00      0.00      0.00         0
 dateOfBirth       1.00      0.19      0.31        27
 dateOfDeath       0.50      0.25      0.33         8
  educatedAt       0.00      0.00      0.00         3
  occupation       1.00      0.56      0.71        54
placeOfBirth       1.00      0.31      

In [38]:
for k in clean_eval_dict.keys():
    print(k)
    devPred = clean_eval_dict[k][0]['devPred']
    devTrue = clean_eval_dict[k][0]['trueDev']

    l_rules = []
    for x, y in zip(devPred, devTrue):
        if x['pred'] != y and y !='Other':
            l_rules.append(x['rule'])
            # print(x['pred'], y,  x['rule'], x['score'])

    c = Counter(l_rules)
    print(c.most_common())


# print()

base_semanticIndex
[('noAnchorMatch', 64), ('noPatternMatch', 20), ('semantic', 3)]
clean_semanticIndex
[('noAnchorMatch', 55), ('noPatternMatch', 23), ('semantic', 2)]


In [39]:
for k in clean_eval_dict.keys():
    print(k)
    testPred = clean_eval_dict[k][0]['testPred']
    trueTest = clean_eval_dict[k][0]['trueTest']

    l_rules = []
    for x, y in zip(testPred, trueTest):
        if x['pred'] != y and y !='Other':
            l_rules.append(x['rule'])
            # print(x['pred'], y,  x['rule'], x['score'])

    c = Counter(l_rules)
    print(c.most_common())


# print()

base_semanticIndex
[('noAnchorMatch', 58), ('noPatternMatch', 22), ('semantic', 6)]
clean_semanticIndex
[('noAnchorMatch', 54), ('noPatternMatch', 21), ('semantic', 2)]
